In [1]:
import pandas as pd
df = pd.read_excel("/mnt/ssd500/hungbnt/DocumentClassification/data/Danh Sách DocID_Updated_by_Samsung.xlsx")
df.abs

ValueError: Excel file format cannot be determined, you must specify an engine manually.